In [1]:
# !pip install import-ipynb

In [2]:
import pandas as pd
import import_ipynb
import preprocessamentofuncoes as pp

importing Jupyter notebook from preprocessamentofuncoes.ipynb


In [3]:
covidAllData = pd.read_csv("../../database/opendatasus/sp-l5-2022.csv", on_bad_lines='skip')
covidAllData.shape

/home/wallece/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (15,20,21,22,30,31,33,34,35,36,45,46,48,50,51,62) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(994104, 65)

In [4]:
# Visualizando Dados
covidAllData.head()

,source_id,dataNotificacao,dataInicioSintomas,sintomas,profissionalSaude,racaCor,outrosSintomas,outrasCondicoes,profissionalSeguranca,cbo,...,codigoFabricanteTeste3,codigoResultadoTeste3,dataColetaTeste3,codigoEstadoTeste4,codigoTipoTeste4,codigoFabricanteTeste4,codigoResultadoTeste4,dataColetaTeste4,idade,Unnamed: 64
0,bhc2Qrr5ml,2022-01-19,2022-01-15,"Coriza, Tosse",Não,Parda,NaN,NaN,Não,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN
1,VC0EW8GtCD,2021-06-01,2021-06-01,"Outros, Assintomático",Não,Parda,TERMINAL DE INTEGRAÇÃO,NaN,Não,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,NaN
2,zENoCK4HiH,2022-01-11,2022-01-09,"Coriza, Febre, Dor de Cabeça",Sim,Branca,NaN,NaN,Não,2233 - Veterinários e zootecnistas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,NaN
3,y0f2iPQwNc,2021-06-01,2021-06-01,"Outros, Assintomático",Não,Parda,TERMINAL DE INTEGRAÇÃO,NaN,Não,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN
4,ETWzw7tEfN,2022-02-08,2022-02-06,"Coriza, Tosse",Não,Branca,NaN,NaN,Não,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.0,NaN


In [5]:
# 'dataPrimeiraDose', 'dataSegundaDose'
colunasImportantes = ['source_id', 'sexo', 'racaCor', 'idade', 'sintomas', 'condicoes', 
                      'classificacaoFinal', 'evolucaoCaso']
covidData = covidAllData[colunasImportantes]

### Limpeza de Dados - Classificação Final


In [6]:
# Visualizando o formato das condições
covidData['classificacaoFinal'].value_counts()

Confirmado Laboratorial              169230
Descartado                           124877
Síndrome Gripal Não Especificada      54484
Confirmado Clínico-Epidemiológico      4042
Confirmado por Critério Clínico        3268
1                                       276
Confirmado Clínico-Imagem               151
2                                        42
Name: classificacaoFinal, dtype: int64

In [7]:
covidData = pp.classFinPreprocessamento(covidData, 'classificacaoFinal')

In [8]:
covidData['classificacaoFinal'].value_counts()

Confirmado    176691
Name: classificacaoFinal, dtype: int64

In [9]:
dados_faltosos = pp.visualizarQntFaltosos(covidData)
dados_faltosos

source_id              0.000000
sexo                   0.000000
racaCor                0.018677
idade                  0.000000
sintomas               0.002264
condicoes             90.805417
classificacaoFinal     0.000000
evolucaoCaso           2.404197
dtype: float64

### Limpeza de Dados - Atibuto Evolução

In [10]:
# covidData[alvo].unique()
evolucao = 'evolucaoCaso'
covidData[evolucao].value_counts()

Cura                        146058
Em tratamento domiciliar     16567
Ignorado                      8217
Cancelado                     1168
Óbito                          265
Internado                      148
Internado em UTI                20
Name: evolucaoCaso, dtype: int64

In [11]:
covidData = pp.evolucaoPreprocessamento(covidData, evolucao)
covidData[evolucao].value_counts()

Cura         162625
Óbito           265
Internado       168
Name: evolucaoCaso, dtype: int64

In [12]:
# dados_faltosos = pp.visualizarQntFaltosos(covidData)
# dados_faltosos

In [13]:
# covidData[evolucao].value_counts()
# covidData.shape

### Limpeza de Dados - Sintomas

In [14]:
# covidData.head()
# covidData.shape

In [15]:
# covidData[alvo].unique()
sintomas = 'sintomas'
covidData = pp.sintomasPreprocessamento(covidData, sintomas)
covidData.shape

(163054, 17)

In [16]:
# covidData.head()
# 'Febre' in covidData['sintomas'] 

### Limpeza de Dados - Condições

In [17]:
# covidData.shape

In [18]:
condicoes = 'condicoes'
covidData = pp.condicoesPreprocessamento(covidData, condicoes)
covidData.shape

(163054, 26)

In [19]:
# pp.condicoesQntValores(covidData)
# covidData.head()

### Limpeza de Dados - Idade

In [20]:
covidData.shape

(163054, 26)

In [21]:
idade = 'idade'
covidData = pp.idadePreprocessamento(covidData, idade, 'faixaetaria')

In [22]:
covidData['faixaetaria'].value_counts()

3.0    51021
2.0    37311
4.0    33734
5.0    16350
0.0    12485
1.0     6883
6.0     4725
7.0      518
Name: faixaetaria, dtype: int64

### Limpeza de Dados - Raça

In [23]:
raca = 'racaCor'
covidData[raca].value_counts()

Branca      86429
Ignorado    38495
Parda       31666
Preta        5060
Amarela      1322
Indigena       25
Branco          1
Name: racaCor, dtype: int64

In [24]:
covidData = pp.racaPreprocessamento(covidData, raca)
covidData[raca].value_counts()

Branca      86429
Parda       31666
Preta        5060
Amarela      1322
Indigena       25
Name: racaCor, dtype: int64

### Limpeza de Dados - Sexo

In [25]:
sexo = 'sexo'
covidData[sexo].value_counts()

Feminino      71148
Masculino     53351
Indefinido        3
Name: sexo, dtype: int64

In [26]:
covidData = pp.sexoPreprocessamento(covidData, sexo)

In [39]:
covidData[sexo].value_counts()

Feminino     71148
Masculino    53351
Name: sexo, dtype: int64

### Limpeza dos Dados - Balanceamento

In [40]:
covidDataBalanced = pp.balanceamento(covidData, 'evolucaoCaso', 'Óbito', 30, 'Cura')
# pp.condicoesQntValoresComAusentes(covidDataBalanced)
covidDataBalanced.shape

(936, 27)

In [41]:
covidDataBalanced = pp.condicoesFormatarValores(covidDataBalanced)
pp.condicoesQntValores(covidDataBalanced)

diabetes
P (41) 	 N (895)
obesidade
P (7) 	 N (929)
renal
P (5) 	 N (931)
respiratoria
P (23) 	 N (913)
imunossupressao
P (7) 	 N (929)
fragilidadeImuno
P (1) 	 N (935)
gestante
P (4) 	 N (932)
cardiaca
P (54) 	 N (882)
puerpera
P (0) 	 N (936)


In [42]:
pp.sintomasQntValores(covidDataBalanced)

assintomatico
P (73) 	 N (863)
febre
P (373) 	 N (563)
dorDeGarganta
P (382) 	 N (554)
dispneia
P (109) 	 N (827)
tosse
P (576) 	 N (360)
coriza
P (387) 	 N (549)
dorDeCabeca
P (414) 	 N (522)
disturbiosGustatorios
P (0) 	 N (936)
disturbiosOlfativos
P (0) 	 N (936)


In [43]:
covidDataBalanced.shape

(936, 27)

### Limpeza dos Dados - Removendo Colunas

In [44]:
colunasRemovidas = ['source_id', 'idade', 'sintomas', 'condicoes', 'classificacaoFinal']
# colunasRemovidas = ['source_id', 'idade', 'sintomas', 'condicoes', 'disturbiosOlfativos', 
#                    'disturbiosGustatorios', 'puerpera', 'fragilidadeImuno', 'gestante', 
#                    'obesidade', 'classificacaoFinal']

In [45]:
covidDataBalanced = pp.removerColunas(covidDataBalanced, colunasRemovidas)
covidDataBalanced.shape

(936, 22)

### Limpeza dos Dados - Formatando Colunas

In [46]:
# Colunas Formatadas: sexo e racaCor
print(covidDataBalanced['sexo'].value_counts(), '\n\n',covidDataBalanced['racaCor'].value_counts())

Feminino     516
Masculino    420
Name: sexo, dtype: int64 

 Branca     662
Parda      219
Preta       44
Amarela     11
Name: racaCor, dtype: int64


In [47]:
covidDataBalanced = pp.formatarColunas(covidDataBalanced)
print(covidDataBalanced['sexo'].value_counts(), '\n\n', covidDataBalanced['racaCor'].value_counts())

0    516
1    420
Name: sexo, dtype: int64 

 0    662
3    219
1     44
2     11
Name: racaCor, dtype: int64


### Salvando a Base Pré-processada

In [48]:
covidDataBalanced.shape

(936, 22)

In [49]:
# dados_faltosos = pp.visualizarQntFaltosos(covidDataBalanced)
# dados_faltosos
covidDataBalanced['evolucaoCaso'].value_counts()

1    720
0    216
Name: evolucaoCaso, dtype: int64

In [50]:
pp.salvarBaseDeDados(covidDataBalanced, 'sp-l5-7030-2022.xlsx')